## Primal problem

\begin{equation}
\begin{aligned}
\underset{\substack{y_t\\ t\in\mathcal{T}}}{\mbox{min.}} & \sum\limits_{t\in\mathcal{T}}\sum\limits_{s\in\mathcal{S}} \sum\limits_{a\in\mathcal{A}} y_t(s, a)c_t(s, a) & (1)\\
\mbox{s.t.} &\sum\limits_{a\in\mathcal{A}} y_{t+1}(s', a) = \sum\limits_{s\in\mathcal{S}}\sum\limits_{a\in\mathcal{A}}\Gamma(s'\mid s, a)y_t(s, a), & (2, 3)\\
&\sum\limits_{a\in\mathcal{A}}y_0(s, a)=p_0(s) & (4)\\
&y_t(s, a)\geq 0 \\
&\forall s, s'\in\mathcal{S}, a\in\mathcal{A}, t\in\mathcal{T}
\end{aligned}\label{Primal MDP}
\end{equation}

In [125]:
''' bchasnov@uw.edu '''
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

########################
##### Setup random problem
########################

tt = 11
n = 10
m = 3

S = list(range(n))
A = list(range(m))
T = list(range(tt))

np.random.seed(0)
p0 = np.random.rand(n)
p0 = p0/p0.sum()

P = np.zeros((n, n, m))
for sp in S:
    for a in A:
        r = np.random.rand(n)
        P[sp, :, a] = r/r.sum()
        
C = np.random.rand(t, n, m)

########################
##### Formulate LP problem
########################

# Primal variable
y = [cp.Variable((n, m)) for t in T] # cvxpy doesn't support tensors

# Cost to minimize (1)
cost = cp.sum([y[t][s, a] * C[t, s, a] for t in T for a in A for s in S])


# Mass propagation (2)
y_tp1 = [cp.sum([y[t][sp, a] 
                 for a in A])         # Sum over actions
         for t in T[1:] for sp in S ] # For time 1 to T and all states

# (3)
Py_t = [cp.sum([P[sp, s, a] * y[t][s, a] 
                for a in A for s in S]) # Sum over actions and state
         for t in T[:-1] for sp in S ]  # For time 0 to T-1 and all future states


# Initial mass (4)
y_0 = [cp.sum([y[0][s, a] for a in A]) for s in S]

objective = cp.Minimize(cost)
constraints = [tmp1 == tmp2 for tmp1, tmp2 in zip(y_tp1, Py_t)] +  \
              [tmp1 == tmp2 for tmp1, tmp2 in zip(y_0, p0)] + \
              [tmp >= 0 for tmp in y]

# Solve
prob = cp.Problem(objective, constraints)
%time result = prob.solve(verbose=True)

-----------------------------------------------------------------
           OSQP v0.3.1  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 330, constraints m = 440
          nnz(P) + nnz(A) = 3990
settings: linear system solver = suitesparse ldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1  -8.1320e+02   1.80e+01   1.66e+01   1.00e-01   6.84e-04s
  75   2.0876e+00   4.44e-04   1.36e-03   1.09e+00   1.83e-03s
plsh   2.0872e+00   4.16e-17   7.77e-16   --------  

## Dual problem
\begin{equation}
\begin{aligned}
\text{maximize} & \sum_{s}V_0(s)p_0(s)  \\
&V_T(s)=c_T(s, a)-\mu_T(s, a),\\
&V_t(s)=c_t(s, a)+\sum\limits_{s'\in\mathcal{S}}\Gamma(s'\mid s, a) V_{t+1}(s') -\mu_t(s, a),
\end{aligned}\label{MDP: vanishing gradient}
\end{equation}


In [139]:
v = [cp.Variable(n) for t in T]
mu = [cp.Variable((n, m)) for t in T]

# Cost to go at time 0
ctg_0 = cp.sum([v[0][s] * p0[s] for s in S])

# Final state
cT_uT = [cp.sum([C[-1, s, a] - mu[-1][s, a] for a in A]) for s in S]

value = [v[t][s] 
         for t in T[:-1] for s in S]
bellman = [np.sum([C[t, s, a] + cp.sum([P[sp, s, a] * v[t+1][sp] for sp in S]) - mu[t][s, a] for a in A])
            for t in T[:-1] for s in S]


constraints = [v[-1][s] == cT_uT[s] for s in S] + \
              [tmp1 == tmp2 for tmp1, tmp2 in zip(value, bellman)]

objective = cp.Maximize(ctg_0)
problem = cp.Problem(objective, constraints)
%time result = problem.solve(verbose=True)

-----------------------------------------------------------------
           OSQP v0.3.1  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 440, constraints m = 110
          nnz(P) + nnz(A) = 1880
settings: linear system solver = suitesparse ldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1  -1.0112e+06   2.72e+00   8.02e+01   1.00e-01   4.47e-04s
  25  -1.0000e+20   8.37e-04   1.38e-01   1.00e-01   7.14e-04s

status:               dual infeasible
number of ite